<a href="https://colab.research.google.com/github/alsnhll/SEIR_COVID19/blob/master/SEIR_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model

### Equations

\begin{equation}
\begin{split}
\dot{S} &= -\beta_1 I_1 S -\beta_2 I_2 S - \beta_3 I_3 S\\
\dot{E} &=\beta_1 I_1 S +\beta_2 I_2 S + \beta_3 I_3 S - a E \\
\dot{I_1} &= a E - \gamma_1 I_1 - p_1 I_1 \\
\dot{I_2} &= p_1 I_1 -\gamma_2 I_2 - p_2 I_2 \\
\dot{I_3} & = p_2 I_2 -\gamma_3 I_3 - \mu I_3 \\
\dot{R} & = \gamma_1 I_1 + \gamma_2 I_2 + \gamma_3 I_3 \\
\dot{D} & = \mu I_3
\end{split}
\end{equation}

### Variables
* $S$: Susceptible individuals
* $E$: Exposed individuals - infected but not yet infectious or symptomatic
* $I_i$: Infected individuals in severity class $i$. Severity increaes with $i$ and we assume individuals must pass through all previous classes
  * $I_1$: Mild infection (hospitalization not required)
  * $I_2$: Severe infection (hospitalization required)
  * $I_3$: Critical infection (ICU required)
* $R$: individuals who have recovered from disease and are now immune
* $D$: Dead individuals
* $N=S+E+I_1+I_2+I_3+R+D$ Total population size (constant)

### Parameters
* $\beta_i$ rate at which infected individuals in class $I_i$ contact susceptibles and infect them
* $a$ rate of progression from the exposed to infected class
* $\gamma_i$ rate at which infected individuals in class $I_i$ recover from disease and become immune
* $p_i$ rate at which infected individuals in class $I_i$ progress to class $I_{I+1}$
* $\mu$ death rate for individuals in the most severe stage of disease

### Basic reproductive ratio

Idea: $R_0$ is the sum of 
1. the average number of secondary infections generated from an individual in stage $I_1$
2. the probability that an infected individual progresses to $I_2$ multiplied by the average number of secondary infections generated from an individual in stage $I_2$
3.  the probability that an infected individual progresses to $I_3$ multiplied by the average number of secondary infections generated from an individual in stage $I_3$

\begin{equation}
\begin{split}
R_0 & = N\frac{\beta_1}{p_1+\gamma_1} + \frac{p_1}{p_1 + \gamma_1} \left( \frac{N \beta_2}{p_2+\gamma_2} + \frac{p_2}{p_2 + \gamma_2} \frac{N \beta_3}{\mu+\gamma_3}\right)\\
&= N\frac{\beta_1}{p_1+\gamma_1} \left(1 + \frac{p_1}{p_2 + \gamma_2}\frac{\beta_2}{\beta_1} \left( 1 + \frac{p_2}{\mu + \gamma_3} \frac{\beta_3}{\beta_2} \right) \right)
\end{split}
\end{equation}

## Modules

In [67]:
# Numerical Computation
import numpy as np, matplotlib.pyplot as plt
from scipy.integrate import odeint

# Custom Modules
import digital_epidemiology as depi

# Reload Custom Modules
from importlib import reload
depi = reload(depi)

## Parameters

Data sources: https://github.com/midas-network/COVID-19/tree/master/parameter_estimates/2019_novel_coronavirus.

In [66]:
# Define parameters based on clinical observations

# DATA SOURCES: https://github.com/midas-network/COVID-19/tree/master/parameter_estimates/2019_novel_coronavirus

IncubPeriod=5     # Incubation period | days
DurMildInf=10     # Duration of mild infections | days
FracMild=0.8      # Fraction of infections that are mild
FracSevere=0.15   # Fraction of infections that are severe
FracCritical=0.05 # Fraction of infections that are critical
CFR=0.02          # Case fatality rate (fraction of infections resulting in death)
TimeICUDeath=7    # Time from ICU admission to death | days
DurHosp=11        # Duration of hospitalization | days

In [57]:
# Define parameters and run ODE

N=1000
b=np.zeros(4) # beta
g=np.zeros(4) # gamma
p=np.zeros(3)

a=1/IncubPeriod

u=(1/TimeICUDeath)*(CFR/FracCritical)
g[3]=(1/TimeICUDeath)-u

p[2]=(1/DurHosp)*(FracCritical/(FracCritical+FracSevere))
g[2]=(1/DurHosp)-p[2]

g[1]=(1/DurMildInf)*FracMild
p[1]=(1/DurMildInf)-g[1]

#b=2e-4*np.ones(4) # all stages transmit equally
b=2.5e-4*np.array([0,1,0,0]) # hospitalized cases don't transmit

#Calculate basic reproductive ratio
R0=N*((b[1]/(p[1]+g[1]))+(p[1]/(p[1]+g[1]))*(b[2]/(p[2]+g[2])+ (p[2]/(p[2]+g[2]))*(b[3]/(u+g[3]))))
print("R0 = {0:4.1f}".format(R0))

R0 =  2.5


In [58]:
print(b)
print(a)
print(g)
print(p)
print(u)

[0.      0.00025 0.      0.     ]
0.2
[0.         0.08       0.06818182 0.08571429]
[0.         0.02       0.02272727]
0.057142857142857134


In [68]:
# Set Parameters 
tmax=365
time=np.arange(0,tmax,0.1)
ic=np.zeros(6)
ic[0]=1

# Compute ODE Solution
solution=odeint(depi.SEIR_ODE,ic,time,args=(b,a,g,p,u,N))
solution=np.hstack((N-np.sum(solution,axis=1,keepdims=True),solution))

# Plot ODE Evolution
depi.plot_SEIR(time, solution, scale = "both")

In [60]:
t1 = 10
t2 = 20

(r,DoublingTime)=depi.growth_rate(time,solution,t1,t2,1)

print("The epidemic growth rate is = {0:4.2f} per day and the doubling time {1:4.1f} days ".format(r,DoublingTime))

The epidemic growth rate is = 0.08 per day and the doubling time  9.0 days 


### Repeat but with a social distancing measure that reduces transmission rate

R0 under intervention =  1.5
The epidemic growth rate is = 0.03 per day and the doubling time 23.3 days 


In [36]:
bSlow=0.6*b
R0Slow=N*((bSlow[1]/(p[1]+g[1]))+(p[1]/(p[1]+g[1]))*(bSlow[2]/(p[2]+g[2])+ (p[2]/(p[2]+g[2]))*(bSlow[3]/(u+g[3]))))

solution_mitigated=odeint(depi.SEIR_ODE,ic,time,args=(bSlow,a,g,p,u,N))
solution_mitigated=np.hstack((N-np.sum(solution_mitigated,axis=1,keepdims=True),solution_mitigated))

# Plot ODE Evolution
depi.plot_SEIR(time, solution_mitigated, 
               scale = "both", linear_legend_loc = "center left", log_legend_loc = "upper left" )

(rSlow,DoublingTimeSlow)=depi.growth_rate(time,solution_mitigated,30,40,1)

plt.show()
print("R0 under intervention = {0:4.1f}".format(R0Slow))
print("The epidemic growth rate is = {0:4.2f} per day and the doubling time {1:4.1f} days ".format(rSlow,DoublingTimeSlow))

#### Compare epidemic growth with and without intervention

In [37]:
# Plot Projections
depi.plot_scenarios(time, scenario1=solution, scenario2=solution_mitigated, 
                    scale="both")

## COVID19 Cases vs Hospital Capacity

Depending on the severity ($I_i$) stage of COVID-19 infection, patients need different level of medical care. 

Individuals in $I_1$ have "mild" infection, meaning they have cough/fever/other flu-like symptoms and may also have mild pneumonia. Mild pneumonia does not require hospitalization, although in many outbreak locations like China and South Korea all symptomatic patients are being hospitalized. This is likely to reduce spread and to monitor these patients in case they rapidly progress to worse outcome. However, it is a huge burden on the health care system.

Individuals in $I_2$ have "severe" infection, which is categorized medically as having any of the following: "dyspnea, respiratory frequency 30/min, blood oxygen saturation 93%, partial pressure of arterial oxygen to fraction of inspired oxygen ratio $<$300, lung infiltrates $>$50% within 24 to 48 hours". These individuals require hospitalization but can be treated on regular wards. They may require supplemental oxygen. 

Individuals in $I_3$ have "critical" infection, which is categorized as having any of the following: "respiratory failure, septic shock, and/or multiple organ dysfunction or failure".
They require ICU-level care, generally because they need mechanical ventilation. 

We consider different scenarios for care requirements. One variation between scenarios is whether we include hospitalization for all individuals or only those with severe or critical infection. Another is the care of critical patients. If ICUs are full, hospitals have protocols developed for pandemic influenza to provide mechanical ventilation outside regular ICU facility and staffing requirements. Compared to "conventional" ventilation protocols, there are "contingency" and "crisis" protocols that can be adopted to increase patient loads. These protocols involve increasing patient:staff ratios, using non-ICU beds, and involving non-critical care specialists in patient care. 



In [38]:
#Parameter sources: https://docs.google.com/spreadsheets/d/1zZKKnZ47lqfmUGYDQuWNnzKnh-IDMy15LBaRmrBcjqE

# All values are adjusted for increased occupancy due to flu season

available_hospital_beds=2.6*(1-0.66*1.1) #Available hospital beds per 1000 ppl in US based on total beds and occupancy
available_ICU_beds=0.26*(1-0.68*1.07) #Available ICU beds per 1000 ppl in US, based on total beds and occupancy. Only counts adult not neonatal/pediatric beds
ConvVentCap=0.062 #Estimated excess # of patients who could be ventilated in US (per 1000 ppl) using conventional protocols
ContVentCap=0.15 #Estimated excess # of patients who could be ventilated in US (per 1000 ppl) using contingency protocols
CrisisVentCap=0.42 #Estimated excess # of patients who could be ventilated in US (per 1000 ppl) using crisis protocols


### Assumptions 1
* Only severe or critical cases go to the hospital
* All critical cases require ICU care and mechanical ventilation


Hospital and ICU beds are filled by COVID19 patients after 69.5 and 74.7 days


In [80]:
# Reload Custom Modules
from importlib import reload
depi = reload(depi)

available_hospital_beds=11  #Available hospital beds per 1000 ppl 
available_ICU_beds=1.5        #Available ICU beds per 1000 ppl 

hospital, ICU, peak_hospital, peak_ICU = depi.plot_H_ICU(time, tmax, solution, available_hospital_beds, available_ICU_beds)


#Find time when hospitalized cases = capacity
icross=np.argmin(np.abs(hospital[0:peak_hospital]-available_hospital_beds)) #find intersection before peak
TimeFillBeds=time[icross]

#Find time when ICU cases = capacity
icross=np.argmin(np.abs(ICU[0:peak_ICU]-available_ICU_beds)) #find intersection before peak
TimeFillICU=time[icross]

plt.show()
print("Hospital and ICU beds are filled by COVID19 patients after {0:4.1f} and {1:4.1f} days".format(TimeFillBeds,TimeFillICU))

Note that we have not taken into account the limited capacity in the model itself. If hospitals are at capacity, then the death rate will increase, since individuals with severe and critical infection will often die without medical care. The transmission rate will probably also increase, since any informal home-care for these patients will likely not include the level of isolation/precautions used in a hospital.

#### Allow for mechanical ventilation outside of ICUs using contingency or crisis capacity

Capacity for mechanical ventilation is filled by COVID19 patients after 31.6 (conventional), 42.7 (contingency) and 56.2 (crisis) days


In [81]:
plt.figure(figsize=(12,7))  # set figure size
plt.plot(time,ICU)
plt.plot(np.array((0, tmax)),ConvVentCap*np.ones(2),linestyle=":")
plt.plot(np.array((0, tmax)),ContVentCap*np.ones(2),linestyle=":")
plt.plot(np.array((0, tmax)),CrisisVentCap*np.ones(2),linestyle=":")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Time (days)", fontsize=15)
plt.ylabel("Number per 1000 People", fontsize=15)
plt.legend(("Cases Needing Mechanical Ventilation","Conventional Capacity","Contingency Capacity","Crisis Capacity"), fontsize=15)
#plt.ylim([0,peak_ICU])
#plt.ylim([0,10])

#Find time when ICU cases = conventional capacity
icrossConv=np.argmin(np.abs(ICU[0:peak_ICU]-ConvVentCap)) #find intersection before peak
TimeConvCap=time[icrossConv]
icrossCont=np.argmin(np.abs(ICU[0:peak_ICU]-ContVentCap)) #find intersection before peak
TimeContCap=time[icrossCont]
icrossCrisis=np.argmin(np.abs(ICU[0:peak_ICU]-CrisisVentCap)) #find intersection before peak
TimeCrisisCap=time[icrossCrisis]

plt.show()
print("Capacity for mechanical ventilation is filled by COVID19 patients after {0:4.1f} (conventional), {1:4.1f} (contingency) and {2:4.1f} (crisis) days".format(TimeConvCap,TimeContCap,TimeCrisisCap))

Compare to the case with intervention

With intervention, hospital and ICU beds are filled by COVID19 patients after 202.3 and 208.3 days


In [95]:
hospital_mitigated=solution_mitigated[:,3]+solution_mitigated[:,4]
ICU_mitigated=solution_mitigated[:,4]

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.plot(time,hospital)
plt.plot(time,hospital_mitigated,color='C0',linestyle="--")
plt.plot(np.array((0, tmax)),available_hospital_beds*np.ones(2),color='C0',linestyle=":")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Time (days)", fontsize=15)
plt.ylabel("Number per 1000 People", fontsize=15)
plt.legend(("Cases Needing Hospitalization","Cases Needing Hospitalization\n(Intervetion)","Available Hospital Beds"), loc="upper right",fontsize=12)
#plt.ylim([0,peakHosp])

plt.subplot(1,2,2)
plt.plot(time,ICU,color='C1')
plt.plot(time,ICU_mitigated,color='C1',linestyle="--")
plt.plot(np.array((0, tmax)),available_ICU_beds*np.ones(2),color='C1',linestyle=":")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Time (days)", fontsize=15)
plt.ylabel("Number per 1000 People", fontsize=15)
plt.legend(("Cases Needing ICU","Cases Needing ICU\n(Intervetion)","Available ICU Beds"), loc="upper right", fontsize=12)
#plt.ylim([0,peakICU])

#Find time when hospitalized cases = capacity
peak_hospital_mitigated=np.argmax(hospital_mitigated) #find peak
icross=np.argmin(np.abs(hospital_mitigated[0:peak_hospital_mitigated]-available_hospital_beds)) #find intersection before peak
TimeFillBedsSlow=time[icross]

#Find time when ICU cases = capacity
peak_ICU_mitigated=np.argmax(ICU_mitigated) #find peak
icross=np.argmin(np.abs(ICU_mitigated[0:peak_ICU_mitigated]-available_ICU_beds)) #find intersection before peak
TimeFillICUSlow=time[icross]

plt.show()
print("With intervention, hospital and ICU beds are filled by COVID19 patients after {0:4.1f} and {1:4.1f} days".format(TimeFillBedsSlow,TimeFillICUSlow))

And for expanded mechanical ventilation capacity

Capacity for mechanical ventilation is filled by COVID19 patients after 54.8 (conventional), 84.8 (contingency) and 122.6 (crisis) days


In [104]:
plt.figure(figsize=(13,6))

plt.plot(time,ICU)
plt.plot(time,ICU_mitigated)
plt.plot(np.array((0, tmax)),ConvVentCap*np.ones(2),linestyle=":")
plt.plot(np.array((0, tmax)),ContVentCap*np.ones(2),linestyle=":")
plt.plot(np.array((0, tmax)),CrisisVentCap*np.ones(2),linestyle=":")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Time (days)", fontsize=15)
plt.ylabel("Number per 1000 People", fontsize=15)
plt.legend(("Cases Needing Mechanical Ventilation","Cases Needing Mechanical Ventilation\n(Intervention)","Conventional Capacity","Contingency Capacity","Crisis Capacity"), loc="upper right", fontsize=14)
#plt.ylim([0,7])

#Find time when ICU cases = conventional capacity (with intervention)
icrossConvSlow=np.argmin(np.abs(NumICUSlow[0:peak_ICU_mitigated]-ConvVentCap)) #find intersection before peak
TimeConvCapSlow=time[icrossConvSlow]
icrossContSlow=np.argmin(np.abs(NumICUSlow[0:peak_ICU_mitigated]-ContVentCap)) #find intersection before peak
TimeContCapSlow=time[icrossContSlow]
icrossCrisisSlow=np.argmin(np.abs(NumICUSlow[0:peak_ICU_mitigated]-CrisisVentCap)) #find intersection before peak
TimeCrisisCapSlow=time[icrossCrisisSlow]

plt.show()
print("Capacity for mechanical ventilation is filled by COVID19 patients after {0:4.1f} (conventional), {1:4.1f} (contingency) and {2:4.1f} (crisis) days".format(TimeConvCapSlow,TimeContCapSlow,TimeCrisisCapSlow))

Interpretation: While interventions that reduce infectiousness do "flatten the curve", cases are still WAY over hospital capacity. There is no way to get anywhere close to staying under hospital bed capacity or mechanical ventilation capacity without reducing $R_0<1$. 

### Assumptions 2
* All cases go to the hospital
* All critical cases require ICU care and mechanical ventilation

Note: No point running this scenario because it would be even more extreme than Assumption 1 (mild cases stayed home) and Assumption 1 already lead to rapid overlow of hospital resources


### Assumptions 3
* Only severe or critical cases go to the hospital
* All critical cases require ICU care and mechanical ventilation
* When hospital capacity is exceed, individual dies

Note: Could be used to simulate expected increases in death if capacity exceeded


## Alternative Models

To be continued, including
* Assuming ~30% of cases are asymptomatic (as seen on Diamond Princess) (This would lead to a re-interpretation of the reported rates of severe and critical infection, so the prevalence of these stages would decrease)
* A parallel instead of series model of disease course (because it is unclear if it is realistic that individuals who pass through the mild stage on the way to a severe state spend as long in the mild stage as individuals who never progress)
* Including pre-symptomatic transmission (for about last ~2 days of exposed period, as estimated in some studies)